### 1. Exploração inicial dos dados

Este notebook (01-exploracao-inicial) foca no entendimento dos dados, análise descritiva inicial, identificação de nulo, ouliners entre outros

In [2]:
# Importação de bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
caminho = '../data/raw/dados_brutos.csv'
df_psico = pd.read_csv(caminho)

In [4]:
df_psico.columns

Index(['ano', 'mes', 'sigla_uf', 'id_municipio', 'id_estabelecimento_cnes',
       'id_estabelecimento_cnes_familia', 'id_procedimento_ambulatorial',
       'id_servico_especializado', 'id_classificacao_servico',
       'data_inicio_atendimento', 'data_termino_atendimento',
       'permanencia_atendimento', 'motivo_saida_permanencia',
       'data_motivo_saida_permanencia', 'ano_processamento',
       'mes_processamento', 'ano_atendimento', 'mes_atendimento',
       'data_nascimento_paciente', 'id_municipio_residencia_paciente',
       'origem_paciente', 'nacionalidade_paciente', 'tipo_idade',
       'idade_paciente', 'sexo_paciente', 'raca_cor_paciente',
       'etnia_paciente', 'carater_atendimento', 'cid_principal_categoria',
       'cid_principal_subcategoria', 'cid_causas_associadas_categoria',
       'cid_causas_associadas_subcategoria', 'tipo_droga', 'destino_paciente',
       'local_realizacao_atendimento', 'indicador_situacao_rua',
       'indicador_estrategia_familia', 'quant

#### 1.1 Junção com dicionários

Os dados originais estão em códigos (id, cid). Nessa etapa carregados os dicionários e os unimos com a tabela principal. As colunas adicionadas na tabela foram:
* Raça e cor do paciente
* Nome do municipio de atendimento
* Nome do municipio de residencia do paciente
* Origem do atendimento do paciente 
* Tipo de idade do paciente
* Destino do paciente após o atendimento

In [5]:
dic_sid = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_cid_10.csv')
dic_municipio = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_municipio.csv')
dic_geral = pd.read_csv('../data/dicionarios/br_ms_sia_dicionario.csv')

In [6]:
print(dic_municipio.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_municipio               5571 non-null   int64  
 1   id_municipio_6             5571 non-null   int64  
 2   id_municipio_tse           5570 non-null   float64
 3   id_municipio_rf            5570 non-null   float64
 4   id_municipio_bcb           5570 non-null   float64
 5   nome                       5571 non-null   object 
 6   capital_uf                 5570 non-null   float64
 7   id_comarca                 5570 non-null   float64
 8   id_regiao_saude            5570 non-null   float64
 9   nome_regiao_saude          5570 non-null   object 
 10  id_regiao_imediata         5571 non-null   int64  
 11  nome_regiao_imediata       5571 non-null   object 
 12  id_regiao_intermediaria    5571 non-null   int64  
 13  nome_regiao_intermediaria  5571 non-null   objec

In [31]:
dic_nomes = dic_municipio.set_index('id_municipio')['nome']

# adiciona nome dos municípios de atendimento para a tabela inicial
df_psico['nome_municipio'] = df_psico['id_municipio'].map(dic_nomes)

# adiciona nome dos municípios de residencia dos pacientes para a tabela inicial
df_psico['nome_municipio_residencia'] = df_psico['id_municipio_residencia_paciente'].map(dic_nomes)


In [32]:
# remoção de colunas que não serão mais usadas
df_psico = df_psico.drop(columns=['id_municipio', 'id_municipio_residencia_paciente'])

In [33]:
dic_geral.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 514
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_tabela           500 non-null    object
 1   nome_coluna         500 non-null    object
 2   chave               500 non-null    object
 3   cobertura_temporal  486 non-null    object
 4   valor               500 non-null    object
dtypes: object(5)
memory usage: 23.4+ KB


In [28]:
colunas_chave = ['nome_coluna', 'chave']
dic_geral = dic_geral.drop_duplicates(subset=colunas_chave, keep='first')

In [29]:
# adiciona a etnia dos pacientes
raca_cor = dic_geral[dic_geral['nome_coluna'] == 'raca_cor_paciente']
mapa = raca_cor.set_index('chave')['valor']
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].astype(str).map(mapa)

# outros traduções que possam ser importantes
origem = dic_geral[dic_geral['nome_coluna'] == 'origem_paciente']
mapa = origem.set_index('chave')['valor']
df_psico['origem_paciente'] = df_psico['origem_paciente'].astype(str).map(mapa)

tipo_idade = dic_geral[dic_geral['nome_coluna'] == 'tipo_idade']
mapa = tipo_idade.set_index('chave')['valor']
df_psico['tipo_idade'] = df_psico['tipo_idade'].astype(str).map(mapa)

destino = dic_geral[dic_geral['nome_coluna'] == 'destino_paciente']
mapa = destino.set_index('chave')['valor']
df_psico['destino_paciente'] = df_psico['destino_paciente'].astype(str).map(mapa)

#### 1.2. Análise exploratória inicial dos dados

In [45]:
df_psico.shape

(3717639, 41)

In [40]:
df_psico.columns

Index(['ano', 'mes', 'sigla_uf', 'id_estabelecimento_cnes',
       'id_estabelecimento_cnes_familia', 'id_procedimento_ambulatorial',
       'id_servico_especializado', 'id_classificacao_servico',
       'data_inicio_atendimento', 'data_termino_atendimento',
       'permanencia_atendimento', 'motivo_saida_permanencia',
       'data_motivo_saida_permanencia', 'ano_processamento',
       'mes_processamento', 'ano_atendimento', 'mes_atendimento',
       'data_nascimento_paciente', 'origem_paciente', 'nacionalidade_paciente',
       'tipo_idade', 'idade_paciente', 'sexo_paciente', 'raca_cor_paciente',
       'etnia_paciente', 'carater_atendimento', 'cid_principal_categoria',
       'cid_principal_subcategoria', 'cid_causas_associadas_categoria',
       'cid_causas_associadas_subcategoria', 'tipo_droga', 'destino_paciente',
       'local_realizacao_atendimento', 'indicador_situacao_rua',
       'indicador_estrategia_familia', 'quantidade_produzida_procedimento',
       'quantidade_aprovada_

In [23]:
df_psico.head()

,ano,mes,sigla_uf,id_estabelecimento_cnes,id_estabelecimento_cnes_familia,id_procedimento_ambulatorial,id_servico_especializado,id_classificacao_servico,data_inicio_atendimento,data_termino_atendimento,...,destino_paciente,local_realizacao_atendimento,indicador_situacao_rua,indicador_estrategia_familia,quantidade_produzida_procedimento,quantidade_aprovada_procedimento,quantidade_atendimentos,quantidade_pacientes,nome_municipio,nome_municipio_residencia
0,2012,11,PR,5322219,NaN,301080046,115,1,2012-11-01,2012-11-30,...,NaN,C,NaN,NaN,1,1,1,1,Curitiba,Curitiba
1,2012,12,PR,5322219,NaN,301080046,115,1,2012-12-01,2012-12-31,...,NaN,C,NaN,NaN,1,1,1,1,Curitiba,Curitiba
2,2012,11,PR,5322219,NaN,301080046,115,1,2012-11-01,2012-11-30,...,NaN,C,NaN,NaN,1,1,1,1,Curitiba,Curitiba
3,2012,12,PR,5708621,0016829,301080046,115,1,2012-12-01,2012-12-31,...,NaN,T,NaN,NaN,1,1,1,1,Curitiba,Curitiba
4,2012,11,PR,5708621,0016829,301080046,115,1,2012-11-01,2012-11-30,...,NaN,C,NaN,NaN,1,1,1,1,Curitiba,Curitiba


In [34]:
df_psico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717639 entries, 0 to 3717638
Data columns (total 41 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   ano                                 int64  
 1   mes                                 int64  
 2   sigla_uf                            object 
 3   id_estabelecimento_cnes             int64  
 4   id_estabelecimento_cnes_familia     object 
 5   id_procedimento_ambulatorial        int64  
 6   id_servico_especializado            int64  
 7   id_classificacao_servico            int64  
 8   data_inicio_atendimento             object 
 9   data_termino_atendimento            object 
 10  permanencia_atendimento             object 
 11  motivo_saida_permanencia            int64  
 12  data_motivo_saida_permanencia       object 
 13  ano_processamento                   int64  
 14  mes_processamento                   int64  
 15  ano_atendimento                     int64  
 16  

CID:

Como os dados de causas associadas estão com um porcentual muito alto de valores nulos, elas não serão incluídas na análise e portanto não precisam obter os valores de seus dicionários. 

As colunas cid_principal_categoria e cid_principal_subcategoria vão ser tratados para futura análise

In [44]:
df_psico[['cid_causas_associadas_categoria', 'cid_causas_associadas_subcategoria', 
          'cid_principal_categoria', 'cid_principal_subcategoria']].isnull().mean().round(2)

cid_causas_associadas_categoria       0.98
cid_causas_associadas_subcategoria    0.94
cid_principal_categoria               0.65
cid_principal_subcategoria            0.35
dtype: float64